In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth',100)
import numpy as np
from IPython.display import display

from google.colab import drive
drive.mount('/content/gdrive')

filepath = "/content/gdrive/My Drive/Colab Notebooks/Projects/emails.csv"
# Read the data into a pandas dataframe called emails
employee = pd.read_csv(filepath)
headers = [header for header in employee.columns]
print("Successfully loaded {} rows and {} columns!".format(employee.shape[0], employee.shape[1]))
print(display(employee.head()))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
import tensorflow_hub as hub

#embedS = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embedL = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

from html.parser import HTMLParser 
import re
parser = HTMLParser()
def process(tweet):
    if pd.isna(tweet):
        return ''
    tweet = parser.unescape(tweet) 
    encode_tweet = tweet.encode('ascii','ignore') 
    decode_tweet = encode_tweet.decode(encoding='UTF-8')
    tweet = tweet.replace('\n', ' ')
    tweet = re.sub(r'https?:\/\/.\S+', "", tweet) 
    tweet = re.sub(r'#', '', tweet) 
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = tweet.replace('e mail', 'email')
    tweet = tweet.replace('e-mail', 'email')
    Apos_dict = {"'s":" is","n't":" not","'m":" am","'ll":" will", 
               "'d":" would","'ve":" have","'re":" are"} 
    for key, value in Apos_dict.items(): 
        if key in tweet: 
            tweet = tweet.replace(key, value) 
    tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)", tweet) if s]) 
    return tweet.replace('--', '').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')
  
employee['msg'] = employee['Message-Body'].apply(process)
employee['subject'] = employee['Subject'].apply(process)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

X1 = employee.msg.progress_apply(lambda x: embedL([x]))
X2 = employee.subject.progress_apply(lambda x: embedL([x]))
X1.shape, X2.shape

X1 = np.vstack(X1)
X2 = np.vstack(X2)
X1.shape, X2.shape

X = np.concatenate([X1, X2], axis=1)
X.shape

(12276, 1024)

In [ ]:
y = pd.get_dummies(employee['X-Folder']).values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras import regularizers

NUM_OF_CLASSES=6

reg = regularizers.l1(5e-06) 

# Neural network
model = Sequential()
#model.add(Dropout(0.2, input_shape=(X_train.shape[1],)))
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dense(512, activation='relu', kernel_regularizer=reg, activity_regularizer=reg))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_regularizer=reg, activity_regularizer=reg))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_regularizer=reg, activity_regularizer=reg))
model.add(Dropout(0.2))
model.add(Dense(NUM_OF_CLASSES, activation='softmax', kernel_regularizer=reg, activity_regularizer=reg))

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

opt = opt = tf.keras.optimizers.Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=15, batch_size=64, callbacks=[reduce_lr]);

Epoch 1/15
154/154 [==============================] - ETA: 0s - loss: 0.1287 - accuracy: 0.9757WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.1287 - accuracy: 0.9757
Epoch 2/15
145/154 [===========================>..] - ETA: 0s - loss: 0.0427 - accuracy: 0.9935WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0427 - accuracy: 0.9934
Epoch 3/15
152/154 [============================>.] - ETA: 0s - loss: 0.0358 - accuracy: 0.9936WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0358 - accuracy: 0.9936
Epoch 4/15
148/154 [===========================>..] - ETA: 0s - loss: 0.0281 - accuracy: 0.9973WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0281 - accuracy: 0.9973
Epoch 5/15
144/154 [===========================>..] - ETA: 0s - loss: 0.0225 - accuracy: 0.9985WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


...

154/154 [==============================] - 1s 5ms/step - loss: 0.0104 - accuracy: 0.9996
Epoch 12/15
145/154 [===========================>..] - ETA: 0s - loss: 0.0094 - accuracy: 0.9997WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0093 - accuracy: 0.9997
Epoch 13/15
152/154 [============================>.] - ETA: 0s - loss: 0.0077 - accuracy: 0.9999WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0077 - accuracy: 0.9999
Epoch 14/15
151/154 [============================>.] - ETA: 0s - loss: 0.0071 - accuracy: 0.9999WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0071 - accuracy: 0.9999
Epoch 15/15
147/154 [===========================>..] - ETA: 0s - loss: 0.0062 - accuracy: 0.9998WARNING:tensorflow:Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr


154/154 [==============================] - 1s 5ms/step - loss: 0.0062 - accuracy: 0.9998


In [ ]:
pred = model.predict_classes(X_test)

true = y_test.argmax(axis=1)
accuracy_score(true, pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


0.9947068403908795

In [ ]:
print(metrics.classification_report(true, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.75      0.46      0.57        13
           2       0.88      0.78      0.82         9
           3       1.00      1.00      1.00       280
           4       0.44      0.78      0.56         9
           5       1.00      1.00      1.00      2144

    accuracy                           0.99      2456
   macro avg       0.68      0.67      0.66      2456
weighted avg       1.00      0.99      0.99      2456



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
